In [98]:
import sqlite3
import json

conn = sqlite3.connect("user_profiles.db")
cursor = conn.cursor()

cursor.execute("""
CREATE TABLE IF NOT EXISTS user_profiles (
    user_id TEXT PRIMARY KEY,
    name TEXT,
    age INTEGER,
    gender TEXT,
    location TEXT,
    preferences TEXT,  -- Store preferences as a comma-separated string
    interaction_history TEXT  -- Store interaction history as JSON string
)
""")

In [99]:
def add_or_update_user_profile(user_id, name, age, gender, location, preferences, interaction_history):
    cursor.execute("SELECT * FROM user_profiles WHERE user_id = ?", (user_id,))
    result = cursor.fetchone()

    if result:
        # Update existing profile
        cursor.execute("""
        UPDATE user_profiles
        SET name = ?, age = ?, gender = ?, location = ?, preferences = ?, interaction_history = ?
        WHERE user_id = ?
        """, (name, age, gender, location, preferences, interaction_history, user_id))
        print(f"Updated profile for User ID: {user_id}")
    else:
        # Insert a new profile
        cursor.execute("""
        INSERT INTO user_profiles (user_id, name, age, gender, location, preferences, interaction_history)
        VALUES (?, ?, ?, ?, ?, ?, ?)
        """, (user_id, name, age, gender, location, preferences, interaction_history))
        print(f"Added new profile for User ID: {user_id}")

    conn.commit()

In [100]:
# Function to update user preferences dynamically
def update_user_preferences(user_id, new_preferences):
    cursor.execute("SELECT preferences FROM user_profiles WHERE user_id = ?", (user_id,))
    result = cursor.fetchone()

    if result:
        # Merge existing preferences with new ones
        existing_preferences = set(result[0].split(", "))
        updated_preferences = existing_preferences.union(set(new_preferences))
        updated_preferences_str = ", ".join(updated_preferences)

        cursor.execute("""
        UPDATE user_profiles
        SET preferences = ?
        WHERE user_id = ?
        """, (updated_preferences_str, user_id))
        print(f"Updated preferences for User ID: {user_id}")
    else:
        print(f"User ID: {user_id} not found!")

    conn.commit()

In [101]:
# Example usage
# Adding a new user
add_or_update_user_profile(
    user_id="001",
    name="John",
    age=25,
    gender="Male",
    location="London",
    preferences="Spa, Italian Cuisine, Adventure Sports",
    interaction_history=json.dumps([{"type": "view", "item": "Yoga Retreat", "timestamp": "2025-01-15"}])
)

# Adding another new user
add_or_update_user_profile(
    user_id="002",
    name="James",
    age=26,
    gender="Male",
    location="New York",
    preferences="Spa, South Indian Cuisine, Games",
    interaction_history=json.dumps([{"type": "view", "item": "Gym", "timestamp": "2025-01-06"}])
)

Added new profile for User ID: 001
Added new profile for User ID: 002


In [102]:
# Query to fetch all records
cursor.execute("SELECT * FROM user_profiles")

# Fetch and display all rows
rows = cursor.fetchall()
for row in rows:
    print(row)

('001', 'John', 25, 'Male', 'London', 'Spa, Italian Cuisine, Adventure Sports', '[{"type": "view", "item": "Yoga Retreat", "timestamp": "2025-01-15"}]')
('002', 'James', 26, 'Male', 'New York', 'Spa, South Indian Cuisine, Games', '[{"type": "view", "item": "Gym", "timestamp": "2025-01-06"}]')


In [103]:
# Adding a new interaction
add_interaction("001", {"type": "purchase", "item": "Spa", "timestamp": "2025-01-05"})
add_interaction("002", {"type": "purchase", "item": "Jacuzzi", "timestamp": "2025-01-10"})

Added interaction for User ID: 001
Added interaction for User ID: 002


In [104]:
# Fetch and print updated user profile
cursor.execute("SELECT * FROM user_profiles WHERE user_id = ?", ("001",))
user_profile = cursor.fetchone()
print("Updated User Profile:", user_profile)

Updated User Profile: ('001', 'John', 25, 'Male', 'London', 'Spa, Italian Cuisine, Adventure Sports', '[{"type": "view", "item": "Yoga Retreat", "timestamp": "2025-01-15"}, {"type": "purchase", "item": "Spa", "timestamp": "2025-01-05"}]')


In [105]:
# Query to fetch all records
cursor.execute("SELECT * FROM user_profiles")

# Fetch and display all rows
rows = cursor.fetchall()
for row in rows:
    print(row)

('001', 'John', 25, 'Male', 'London', 'Spa, Italian Cuisine, Adventure Sports', '[{"type": "view", "item": "Yoga Retreat", "timestamp": "2025-01-15"}, {"type": "purchase", "item": "Spa", "timestamp": "2025-01-05"}]')
('002', 'James', 26, 'Male', 'New York', 'Spa, South Indian Cuisine, Games', '[{"type": "view", "item": "Gym", "timestamp": "2025-01-06"}, {"type": "purchase", "item": "Jacuzzi", "timestamp": "2025-01-10"}]')


In [106]:
conn.close()